In [1]:
def params_to_label(input_params):
    """ 
    
    Convert list of input parameters into a label for saving files easily
    
    e.g. [0, 1, 1.5, 0.2] -> 0_1_15_02
    """
    
    test = str(input_params).replace(".", "")
    test2 = str(test).replace(", ", "_")
    test3 = str(test2).replace("[", "")
    test4 = str(test3).replace("]", "")
    label = test4
    return label



In [2]:
def J_UD(w, aud, Gamma, w0):
    """ 
    
    Underdamped spectral density 
    
    Inputs:
    w (frequency), 
    aud (coupling strength), 
    Gamma (peak width), 
    w0 (peak centre)
    
    Output:
    J(w) (spectral density for these parameters)
    """
    
    w *= conv
    num = (aud/pi) * Gamma * (w0**2) * w
    den = (w0**2 - w**2)**2 + (Gamma*w)**2
    ans = num/den
    
    return ans 

In [3]:
def map_parameters(aud, Gamma, w0):
    """ 
    
    Takes in the unmapped parameters for the underdamped spectral density 
    and gives back the mapped parameters for the RC Hamiltonian 
            
    Inputs:
    aud (coupling strength),
    Gamma (peak width),
    w0 (peak centre)
    
    Outputs: 
    Omega (RC frequency),
    llambda (TLS-RC coupling strength),
    gam (RC-RB coupling strength)
    """
    
    Omega = w0                     # RC frequency
    llambda = np.sqrt(aud*w0/2)    # S-RC coupling strength
    gam = Gamma/(2*pi*w0)          # residual bath coupling strength

    print(f'Omega = {Omega:.3f}, lambda = {llambda:.3f}, gamma = {gam:.3f}')
    return Omega, llambda, gam

In [4]:
def N(w, beta):
    """ boson thermal distribution """
    ans = 1/(np.exp(beta*w)-1)
    return ans

def J(w, gam):
    """ 
    
    mapped spectral density - Ohmic with no cutoff
    
    Inputs:
    w (frequency),
    gam (RC-RB coupling strength)
    
    Outputs:
    J (spectral density)
    
    """
    return gam*w

def L(chi, A, vals, vecs, beta, gam):
    """ 
    
    Chi dependent liouvillian for the reaction coordinate master equation
    
    Inputs:
    chi (counting field parameter),
    A (interaction operator)
    vals (eigenvales of H_ES),
    vecs (eigenvectors of H_ES),
    beta (inverse bath temperature),
    gam (RC-RB coupling strength)
    
    Outputs:
    L (Liouvillian superoperator for this value of chi)
    
    """
    
    def A_jk(j, k, op):
        """ 
        jk^th matrix elment of A in the extended system Hamiltonian basis
        
        Inputs:
        j,k (indexes)
        op (operator to take the matrix element of)
        
        Outputs
        ajk (matrix element of op wrt j and k indexed eigenvectors)
        
        """
        ajk = vecs[j].dag() * op * vecs[k]
        return ajk

    # initialise the rate operators to zero
    A1 = 0
    A2 = 0
    A3 = 0
    A4 = 0
    
    for m in range(2*M):
        for n in range(2*M):

            # difference in extended system eigenvalues
            l_mn = vals[m]-vals[n]
            
            # outer product of states
            mn = vecs[m]*vecs[n].dag() # |lambda_m> <lambda_n|

            # mn^th element of A(chi)
            A_mn = A_jk(m ,n, A)

            # add to the different rate operators depending on the difference in eigenvalues
            if l_mn > 0:
                
                occ = N(l_mn, beta)
                
                J_mn = J(l_mn, gam)
                
                A1 += pi * A_mn  * J_mn * occ * mn
                A2 += pi * A_mn  * J_mn * ( 1 + occ) * np.exp(1j * chi * l_mn) * mn 
                A3 += pi * A_mn  * J_mn * occ * np.exp(-1j * chi * l_mn) * mn
                A4 += pi * A_mn  * J_mn * ( 1 + occ) * mn
            
            elif l_mn < 0:
                
                l_mn = np.abs(l_mn)
                
                occ = N(l_mn, beta)
                
                J_mn = J(l_mn, gam)

                A1 += pi * A_mn  * J_mn * ( 1 + occ ) * mn 
                A2 += pi * A_mn  * J_mn * occ * np.exp(-1j * chi * l_mn) * mn
                A3 += pi * A_mn  * J_mn * ( 1 + occ ) * np.exp(1j * chi * l_mn) * mn
                A4 += pi * A_mn  * J_mn * occ * mn
            
            else:
                val = pi * A_mn  * (gam / beta) * mn
                A1 += val
                A2 += val
                A3 += val
                A4 += val
    
    #create the different Liouvillian terms from the rate operators
    L_coh = -1j*(qutip.spre(H_ES) - qutip.spost(H_ES))
    L_1 = qutip.spre(A * A1)
    L_2 = qutip.sprepost(A, A2)
    L_3 = qutip.sprepost(A3, A)
    L_4 = qutip.spost(A4 * A)
    
    #create the total Liouvillian
    L_tot = L_coh - L_1 + L_2 + L_3 - L_4
    
    return L_tot

In [5]:
def thermal_state(H, beta):
    """
    Create a thermal state
    
    Inputs:
    H (Hamiltonian)
    beta (inverse temperature)
    
    Outputs
    rho (thermal state wrt the Hamiltonian and inverse temperature)
    
    """
    
    Z = ((-(beta)*(H)).expm()).tr() # partition function
    rho = (-beta*H).expm()/Z        # thermal state 
    return rho

def CFD_IC(rho_s_0, chi, beta, H):
    """ 
    
    CFD initial condition for master equation
    
    Inputs:
    rho_s_0 (initial state of TLS)
    chi (counting field parameter)
    beta (inverse bath temperature)
    
    Outputs
    rho0 (counting field dependent initial state)
    
    Notes:
    For a standard initial state use \chi=0, giving the TLS and RC thermal state in a product
    H will be the RC self Hamiltonian \Omega a.dag() a
    
    """
    
    rho_RC = thermal_state(H, beta)          # thermal state 
    E = (-0.5 * chi * 1j * H).expm()         # chi dependent shift
    rho0 = qutip.tensor(rho_s_0, E*rho_RC*E) # initial state for ES
    return rho0

def mike_trace(item):
    """
    Trace of a matrix that doesn't throw away complex entries """
    dim = 2*M
    total=0
    for i in range(dim):
        total+=item[i][i]
    return total

In [6]:
def coherence_sz(t, rho_s_0, aud, Gamma, w0, beta, eps):
    """ 
    
    Analytic coherence for sigmaz coupling 
    
    Inputs:
    t (time)
    rho_s_0 (initial state of TLS)
    aud (coupling strength),
    Gamma (peak width),
    w0 (peak center),
    beta (inverse temperature)
    eps (spin splitting)
    
    Outputs
    sx (coherence)
    
    """
    
    def r_integrand(w, aud, Gamma, w0, beta):
        aux0 = J_UD(w, aud, Gamma, w0)/(w**2) 
        aux1 = (1-np.cos(w*t))
        aux2 = 1/np.tanh(beta*w/2)
        ans = aux0*aux1*aux2
        return ans 

    rho01_0 = rho_s_0.full()[0][1].real
    rho10_0 = rho_s_0.full()[1][0].real
    
    integral = quad(r_integrand, 0, np.inf, args=(aud, Gamma, w0, beta), limit=int_limit)[0]    
    
    sx = rho01_0*np.exp(-4*integral)*np.exp(1j*(eps)*t) 
    sx += rho10_0 * np.exp(-4*integral)*np.exp(-1j*(eps)*t)
    
    return sx

In [ ]:
def rev(cf):
    """ 
    extend a characteristic function to negative values of chi 
    
    Inputs:
    cf (characteristic function in list)
    
    Outputs:
    revcf (characteristic function extended to negative chi)
    
    
    """
    
    rev_list = np.conj(np.flip(cf))
    rev_list = np.delete(rev_list,-1)
    revcf = np.concatenate((rev_list, cf))
    return revcf

In [1]:
# analytic form of full bath heat characteristic function

def cf_int_real(w, aud, Gamma, w0, beta, t, chi):
    """ 
    
    Real part of the integral for the exact characteristic function 
    for independent boson model 
    
    Inputs:
    w (frequency),
    aud (coupling strength),
    Gamma (peak width),
    w0 (peak center), 
    beta (inverse bath temperature), 
    t (time), 
    chi (counting field parameter)
    
    
    Outputs
    Integrand 
    
    """
    
    aux0 = J_UD(w, aud, Gamma, w0)/(w**2)
    aux1 = (1-np.cos(w*t))
    aux3 = 1/np.tanh(beta*w/2)*(1-np.cos(w*chi))
    
    integrand = aux0 * aux1 * aux3
    
    return integrand

def cf_int_imag(w, aud, Gamma, w0, t, chi):
    """ 
    
    Imag part of the integral for the exact characteristic function 
    for independent boson model 
    
    Inputs:
    w (frequency),
    aud (coupling strength),
    Gamma (peak width),
    w0 (peak center), 
    beta (inverse bath temperature), 
    t (time), 
    chi (counting field parameter)
    
    
    Outputs
    Integrand 
    
    """
    
    aux0 = J_UD(w, aud, Gamma, w0)/(w**2)
    aux1 = (1-np.cos(w*t))
    aux4 = np.sin(w*chi)
    
    integrand = aux0 * aux1 * aux4
   
    return integrand

In [2]:
def exact_mean(aud, Gamma, w0, t):
    """
    
    Analytic expression for mean heat transfer in IBM (full bath)
    
    Inputs:
    aud (coupling strength), 
    Gamma (peak width), 
    w0 (peak center), 
    t (time)
    
    
    Outputs:
    mean (mean heat transfer)
    """
    
    def exact_mean_integrand(w, aud, Gamma, w0, t):
        aux0 = J_UD(w, aud, Gamma, w0)/w
        aux1 = (1-np.cos(w*t))
        ans = aux0*aux1
        return ans

    integral = quad(exact_mean_integrand, epsilon, cutoff, args=(aud, Gamma, w0, t), epsabs=epsilon, epsrel=epsilon, limit=int_limit,)[0]
    mean = 2*integral
    return mean

def exact_var(aud, Gamma, w0, beta, t):
    """
    
    Analytic expression for mean heat transfer in IBM (full bath)
    
    Inputs:
    aud (coupling strength), 
    Gamma (peak width), 
    w0 (peak center), 
    beta (inverse bath temperature)
    t (time)
    
    
    Outputs:
    var (variance of heat transfer)
    """
    
    def exact_var_integrand(w, aud, Gamma, w0, beta, t):
        aux0 = J_UD(w, aud, Gamma, w0)
        aux1 = (1-np.cos(w*t))
        aux2 = 1/np.tanh(beta*w/2)
        ans = aux0*aux1*aux2
        return ans
    
    integral = quad(exact_var_integrand, epsilon, cutoff, args=(aud, Gamma, w0, beta, t),epsabs=epsilon, epsrel=epsilon, limit=int_limit,)[0]
    var = 2*integral
    return var

In [16]:
# mean/variance from CF using finite difference method

def mean_var_heat(chi_eps, cf):
    """ Calculate mean and variance, given a small value of chi and the characteristic function """
    cf  = np.array(cf)
    mean = cf.imag/chi_eps
    var = 2 - 2*cf.real - (cf.imag)**2
    var /= chi_eps**2 
    return mean, var
